In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import open3d as o3d

In [2]:
latitude = 3.9525536
longitude = 50.4545097
size = 100

In [3]:
p = gpd.GeoSeries([Point(latitude, longitude)])
p.crs = 'epsg:4326'
p = p.to_crs(epsg=31370)

x = p[0].x
y = p[0].y

In [4]:
hainaut = pd.read_csv('./hainaut.csv')

In [5]:
place = hainaut[(hainaut["x"] <= x + size) & 
            (hainaut["x"] >= x - size) & 
            (hainaut["y"] <= y + size) & 
            (hainaut["y"] >= y - size)]

In [6]:
place

,x,y,z
949961,120344.903332,127057.602803,58.190000
949962,120345.903332,127057.602803,58.200000
949963,120346.903332,127057.602803,58.190000
949964,120347.903332,127057.602803,58.240000
949965,120348.903332,127057.602803,58.190000
...,...,...,...
1244876,120540.903332,127256.602803,55.447870
1244877,120541.903332,127256.602803,55.741962
1244878,120542.903332,127256.602803,56.250000
1244879,120543.903332,127256.602803,54.480010


In [7]:
meanx = place['x'].mean()
meany = place['y'].mean()
minz = place['z'].min()

final = pd.DataFrame()
final['x'] = place['x'] - meanx
final['y'] = place['y'] - meany
final['z'] = place['z'] - minz

In [8]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(final.to_numpy())
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=8, linear_fit =True)

densities = np.asarray(densities)
density_colors = plt.get_cmap('viridis')(
    (densities - densities.min()) / (densities.max() - densities.min()))
density_colors = density_colors[:, :3]
density_mesh = o3d.geometry.TriangleMesh()
density_mesh.vertices = poisson_mesh.vertices
density_mesh.triangles = poisson_mesh.triangles
density_mesh.triangle_normals = poisson_mesh.triangle_normals
density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)

vertices_to_remove = densities < np.quantile(densities, 0.01)
density_mesh.remove_vertices_by_mask(vertices_to_remove)

[Open3D DEBUG] Input Points / Samples: 40200 / 40200
[Open3D DEBUG] #   Got kernel density: 0.0378718 (s), 208.719 (MB) / 208.719 (MB) / 227 (MB)
[Open3D DEBUG] #     Got normal field: 0.195382 (s), 226.703 (MB) / 226.703 (MB) / 227 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 5.491606e-05 / 2.207626e+00
[Open3D DEBUG] #       Finalized tree: 0.229609 (s), 251.969 (MB) / 251.969 (MB) / 251 (MB)
[Open3D DEBUG] #  Set FEM constraints: 1.32391 (s), 251.969 (MB) / 251.969 (MB) / 251 (MB)
[Open3D DEBUG] #Set point constraints: 0.133619 (s), 251.969 (MB) / 251.969 (MB) / 251 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 596037 / 652440 / 28745
[Open3D DEBUG] Memory Usage: 251.969 MB
[Open3D DEBUG] # Linear system solved: 24.4348 (s), 281.137 (MB) / 281.137 (MB) / 281 (MB)
[Open3D DEBUG] Got average: 0.1341 (s), 281.137 (MB) / 281.137 (MB) / 281 (MB)
[Open3D DEBUG] Iso-Value: 5.022038e-01 = 2.018859e+04 / 4.020000e+04
[Open3D DEBUG] #          Total Solve:      39.7 (s), 

NameError: name 'plt' is not defined

In [ ]:
o3d.io.write_triangle_mesh("./3D/3D_houses.obj", density_mesh)

In [ ]:
o3d.__version__

In [ ]:
import sklearn

In [ ]:
sklearn.__version__